In [1]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
import numpy as np
import pandas as pd

In [ ]:
gis_file = 'D:/2Gis/杭州基础数据库.gdb'
xzyd = gpd.read_file(gis_file,layer='L1现状用地xzyd_c4_rk')
zone = gpd.read_file(gis_file,layer='Z1市区交通小区3143')
road = gpd.read_file(gis_file,layer='R1路网中心线_框架数据')
building = gpd.read_file('D:\\2Gis\\Arcgis Packages\\市区建筑\\commondata\\4-gis\\市区建筑.shp')

In [7]:
xzyd = xzyd.to_crs("EPSG:4549")
zone = zone.to_crs("EPSG:4549")
road  = road.to_crs("EPSG:4549")
building = building.to_crs("EPSG:4549")

In [ ]:
zone.explore()

### 2、面层间的几个计算函数

In [ ]:
# # Analysis for each land-block分析面层内落入的点数量、线长度、面积和等（不分割）
# def analyze_land(land_geometry):
#     # Count features intersecting the neighborhood boundary
#     num_schools = schools[schools.geometry.intersects(neighborhood_geometry)].shape[0]
#     num_subways = subways[subways.geometry.intersects(neighborhood_geometry)].shape[0]
#     bike_path_length = bike_paths[bike_paths.geometry.intersects(neighborhood_geometry)].length.sum()
#     park_area = parks[parks.geometry.intersects(neighborhood_geometry)].area.sum()

#     return num_schools, num_subways, bike_path_length, park_area

In [4]:
def analyze_length(land_geometry):
    # 地块内，道路长度计算
    # 1. 筛选出与地块相交的自行车道（这里的地块只有一块）
    intersecting_road = road[road.geometry.intersects(land_geometry)].copy()    
    road_length = 0
    
    if not intersecting_road.empty:
        # 2. 计算自行车道与邻里几何形状的交集
        # intersection() 方法会返回 LineString 的相交部分
        # (如果 LineString 与 Polygon 相交，结果通常是 LineString 或 MultiLineString)
        intersected_geometries = intersecting_road.geometry.intersection(land_geometry)
        
        # 3. 对相交后的几何图形计算长度并求和
        # 这一步只计算几何图形位于 land_geometry 内部的那一部分的长度
        road_length = intersected_geometries.length.sum()

    # park_area 的计算现在是 jzmj 的折减求和 (沿用您之前的要求)
    # ... (park_area 的计算逻辑与您上一个问题中的答案相同)
    
    return road_length

In [ ]:
# 两个面层，根据相交部分面积占比，计算属性和
def calculate_proportional_park_attribute_fast(
    neighborhoods: gpd.GeoDataFrame,
    parks: gpd.GeoDataFrame,
    attribute_col: str = 'BUILDAREA'
):
    if neighborhoods.crs != parks.crs:
        neighborhoods = neighborhoods.to_crs(parks.crs)

    print("Step 1/3: Performing spatial join...")
    neighborhoods_for_join = neighborhoods[['geometry']].copy()
    # ensure a stable, named index
    if neighborhoods_for_join.index.name is None:
        neighborhoods_for_join = neighborhoods_for_join.rename_axis('dk_id')
    right_index_name = neighborhoods_for_join.index.name  # e.g., 'dk_id'

    neighborhoods_for_join = neighborhoods_for_join.rename_geometry('neighborhood_geometry')

    joined_df = gpd.sjoin(
        parks.copy(),
        neighborhoods_for_join,
        how='inner',
        predicate='intersects',
        lsuffix='park',
        rsuffix='neigh',
    )

    # --- Find the right-side id column (can be 'index_right' OR the right index name, e.g. 'dk_id') ---
    candidates = []
    if 'index_right' in joined_df.columns:
        candidates.append('index_right')
    if right_index_name and right_index_name in joined_df.columns:
        candidates.append(right_index_name)
    # common fallback some users see:
    if 'index_neigh' in joined_df.columns:
        candidates.append('index_neigh')

    if not candidates:
        raise KeyError(
            f"Could not find right-side id column after sjoin. "
            f"Available columns: {list(joined_df.columns)}; expected one of "
            f"['index_right', '{right_index_name}', 'index_neigh']"
        )
    right_id_col = candidates[0]  # pick the first match

    # Bring back right-hand geometries aligned by the detected id column
    neigh_geom_aligned = neighborhoods_for_join.loc[
        joined_df[right_id_col], 'neighborhood_geometry'
    ].values
    joined_df = joined_df.reset_index(drop=False)
    joined_df['neighborhood_geometry'] = neigh_geom_aligned

    print("Step 2/3: Calculating intersection area and proportional attribute...")

    joined_df['intersection_geometry'] = joined_df['geometry'].intersection(
        joined_df['neighborhood_geometry']
    )
    joined_df['original_area']   = joined_df['geometry'].area
    joined_df['intersect_area']  = joined_df['intersection_geometry'].area
    joined_df['area_ratio'] = np.where(
        joined_df['original_area'] > 0,
        joined_df['intersect_area'] / joined_df['original_area'],
        0.0
    )
    joined_df['area_ratio'] = np.clip(joined_df['area_ratio'], 0.0, 1.0)
    joined_df['discounted_attribute'] = joined_df[attribute_col] * joined_df['area_ratio']

    print("Step 3/3: Grouping and summing the results...")

    results = (
        joined_df
        .groupby(right_id_col)['discounted_attribute']
        .sum()
        .rename(f'sum_weighted_{attribute_col}')
        .to_frame()
    )

    # Optional: align to all neighborhoods (fill missing with 0)
    results = results.reindex(neighborhoods_for_join.index, fill_value=0.0)

    return results


### 计算xzyd图层中building的建筑面积和

In [8]:
# Apply analysis to each neighborhood
zone['road_length'] = zone.geometry.apply(
    lambda geom: pd.Series(analyze_length(geom))
)

In [17]:
# # 运行函数
weighted_jzmj_results = calculate_proportional_park_attribute_fast(xzyd, building, attribute_col='BUILDAREA')
print(weighted_jzmj_results.head())

Step 1/3: Performing spatial join...
Step 2/3: Calculating intersection area and proportional attribute...
Step 3/3: Grouping and summing the results...
       sum_weighted_BUILDAREA
dk_id                        
0                    0.000000
1                    0.000000
2                    0.000000
3                  223.748551
4                   75.739186


### 3、导出gis，保存至gdb

In [21]:
# 将结果合并到原始 xzyd 图层
xzyd_with_attr = xzyd.merge(
    weighted_jzmj_results,
    how='left',         # 保留所有邻里
    left_index=True,    # 使用邻里索引匹配
    right_index=True
)

# 如果某些邻里没有交集，则填 0
xzyd_with_attr['sum_weighted_BUILDAREA'] = xzyd_with_attr['sum_weighted_BUILDAREA'].fillna(0)

# 查看结果
xzyd_with_attr.head()

,BSM,地块编号,xzdkID,DLBM,DLMC,XDLDM,XDLMC,dlmc_12,现状地类代码,现状地类名称,...,现状人口数adj,现状岗位数adj,岗位大类,用地类别f_规划,岗位大类_规划,Shape_Length,Shape_Area,面积,geometry,sum_weighted_BUILDAREA
0,330111211000021113,None,0.0,0702,农村宅基地,0702,农村宅基地,None,None,None,...,4.274632,NaN,E7居住类,None,None,0.001429,1.035901e-07,1107.401123,"MULTIPOLYGON Z (((487958.297 3325884.913 0, 48...",0.000000
1,330111211000021395,None,1.0,0702,农村宅基地,0702,农村宅基地,教育用地,None,None,...,1.725805,NaN,E7居住类,None,None,0.000734,3.346675e-08,357.676483,"MULTIPOLYGON Z (((489058.323 3328737.046 0, 48...",0.000000
2,330111211000021815,None,2.0,0702,农村宅基地,0702,农村宅基地,二类居住用地,None,None,...,22.750396,NaN,E7居住类,None,None,0.003064,4.408419e-07,4715.070312,"MULTIPOLYGON Z (((497077.255 3320251.262 0, 49...",0.000000
3,330111211000021902,None,3.0,0702,农村宅基地,0702,农村宅基地,None,None,None,...,1.079596,NaN,E7居住类,None,None,0.000904,4.610054e-08,492.982025,"MULTIPOLYGON Z (((491655.224 3322328.105 0, 49...",223.748551
4,330111211000022349,None,4.0,0702,农村宅基地,0702,农村宅基地,None,None,None,...,1.523594,NaN,E7居住类,None,None,0.000849,3.689932e-08,394.708130,"MULTIPOLYGON Z (((486070.291 3318900.24 0, 486...",75.739186


In [23]:
xzyd_with_attr.to_file(
    gis_file,
    layer='L1现状用地xzyd_c4_rk251016',
    driver="OpenFileGDB"
)